# Phase 1: Data Exploration

**Objective:** Understand ERCOT electricity market data, renewable generation patterns, and relationships between variables.

**Author:** Amalie Berg  
**Date:** October 2025

---

## Contents
1. [Setup and Data Loading](#setup)
2. [Price Analysis](#prices)
3. [Load and Renewable Generation](#generation)
4. [Weather Data](#weather)
5. [Feature Relationships](#relationships)
6. [Time Series Characteristics](#timeseries)
7. [Key Insights](#insights)

## 1. Setup and Data Loading <a id='setup'></a>

In [ ]:
# Import libraries
import sys
sys.path.append('..')  # Add parent directory to path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from src import data_processing, visualization

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

### Load Data

First, let's check if data exists. If not, we'll generate it.

In [ ]:
# Check if processed data exists
data_path = Path('../data/processed/merged_data.csv')

if not data_path.exists():
    print("Processed data not found. Running data collection...")
    df = data_processing.main()
else:
    print("Loading existing processed data...")
    df = pd.read_csv(data_path, parse_dates=['datetime'])

print(f"\nData loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")

### Initial Data Inspection

In [ ]:
# Display first few rows
print("First 5 rows:")
df.head()

In [ ]:
# Data types and missing values
print("\nData Info:")
df.info()

In [ ]:
# Summary statistics
print("\nSummary Statistics:")
df.describe()

## 2. Price Analysis <a id='prices'></a>

Let's analyze ERCOT Day-Ahead Market (DAM) electricity prices.

In [ ]:
# Price time series
visualization.plot_price_timeseries(
    df, 
    title='ERCOT Day-Ahead Market Prices (2023-2024)',
    save_path='price_timeseries.png'
)
plt.show()

In [ ]:
# Price distribution
visualization.plot_price_distribution(
    df,
    title='DAM Price Distribution',
    save_path='price_distribution.png'
)
plt.show()

In [ ]:
# Price statistics
print("Price Statistics:")
print(f"Mean: ${df['dam_price'].mean():.2f}/MWh")
print(f"Median: ${df['dam_price'].median():.2f}/MWh")
print(f"Std Dev: ${df['dam_price'].std():.2f}/MWh")
print(f"Min: ${df['dam_price'].min():.2f}/MWh")
print(f"Max: ${df['dam_price'].max():.2f}/MWh")
print(f"\n95th Percentile: ${df['dam_price'].quantile(0.95):.2f}/MWh")
print(f"5th Percentile: ${df['dam_price'].quantile(0.05):.2f}/MWh")

# Identify price spikes (>95th percentile)
spike_threshold = df['dam_price'].quantile(0.95)
spikes = df[df['dam_price'] > spike_threshold]
print(f"\nNumber of price spikes (>95th %ile): {len(spikes)} hours ({len(spikes)/len(df)*100:.2f}%)")

### Temporal Patterns

In [ ]:
# Hourly patterns
visualization.plot_hourly_patterns(
    df,
    value_col='dam_price',
    title='Average Price by Hour of Day',
    save_path='price_hourly_pattern.png'
)
plt.show()

In [ ]:
# Seasonal patterns
visualization.plot_seasonal_patterns(
    df,
    value_col='dam_price',
    title='Average Price by Month',
    save_path='price_seasonal_pattern.png'
)
plt.show()

## 3. Load and Renewable Generation <a id='generation'></a>

In [ ]:
# Plot load and generation over time
fig, ax = plt.subplots(figsize=(14, 6))

# Sample every 24 hours for cleaner plot
df_daily = df.iloc[::24]

ax.plot(df_daily['datetime'], df_daily['system_load_mw'], 
        label='System Load', alpha=0.7, linewidth=1)
ax.plot(df_daily['datetime'], df_daily['wind_generation_mw'], 
        label='Wind Generation', alpha=0.7, linewidth=1)
ax.plot(df_daily['datetime'], df_daily['solar_generation_mw'], 
        label='Solar Generation', alpha=0.7, linewidth=1)

ax.set_xlabel('Date')
ax.set_ylabel('Power (MW)')
ax.set_title('System Load and Renewable Generation', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/load_generation_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Renewable penetration statistics
print("Renewable Generation Statistics:")
print(f"\nWind Generation:")
print(f"  Mean: {df['wind_generation_mw'].mean():.0f} MW")
print(f"  Max: {df['wind_generation_mw'].max():.0f} MW")

print(f"\nSolar Generation:")
print(f"  Mean: {df['solar_generation_mw'].mean():.0f} MW")
print(f"  Max: {df['solar_generation_mw'].max():.0f} MW")

print(f"\nRenewable Penetration:")
print(f"  Mean: {df['renewable_penetration'].mean()*100:.1f}%")
print(f"  Max: {df['renewable_penetration'].max()*100:.1f}%")

## 4. Weather Data <a id='weather'></a>

In [ ]:
# Weather patterns
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Temperature
axes[0].plot(df['datetime'], df['temperature_f'], linewidth=0.5, alpha=0.7)
axes[0].set_ylabel('Temperature (°F)')
axes[0].set_title('Temperature Over Time', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Solar irradiance
axes[1].plot(df['datetime'], df['solar_irradiance_w_m2'], linewidth=0.5, alpha=0.7, color='orange')
axes[1].set_ylabel('Solar Irradiance (W/m²)')
axes[1].set_title('Solar Irradiance Over Time', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Wind speed
axes[2].plot(df['datetime'], df['wind_speed_ms'], linewidth=0.5, alpha=0.7, color='green')
axes[2].set_ylabel('Wind Speed (m/s)')
axes[2].set_xlabel('Date')
axes[2].set_title('Wind Speed Over Time', fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/weather_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Feature Relationships <a id='relationships'></a>

In [ ]:
# Select key features for correlation analysis
key_features = [
    'dam_price', 'system_load_mw', 'wind_generation_mw', 
    'solar_generation_mw', 'total_renewable_mw',
    'temperature_f', 'solar_irradiance_w_m2', 'wind_speed_ms',
    'renewable_penetration', 'hour', 'is_weekend'
]

visualization.plot_correlation_heatmap(
    df[key_features],
    title='Feature Correlation Matrix',
    save_path='correlation_heatmap.png'
)
plt.show()

In [ ]:
# Price vs Load
visualization.plot_scatter_with_regression(
    df.sample(n=min(5000, len(df))),  # Sample for cleaner plot
    'system_load_mw',
    'dam_price',
    title='Price vs System Load',
    save_path='price_vs_load.png'
)
plt.show()

In [ ]:
# Price vs Renewable Penetration
visualization.plot_scatter_with_regression(
    df.sample(n=min(5000, len(df))),
    'renewable_penetration',
    'dam_price',
    title='Price vs Renewable Penetration',
    save_path='price_vs_renewables.png'
)
plt.show()

## 6. Time Series Characteristics <a id='timeseries'></a>

Check for stationarity, autocorrelation, and other time series properties.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

# Augmented Dickey-Fuller test for stationarity
adf_result = adfuller(df['dam_price'].dropna())
print("Augmented Dickey-Fuller Test for Price:")
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")
print(f"Critical Values:")
for key, value in adf_result[4].items():
    print(f"  {key}: {value:.3f}")

if adf_result[1] < 0.05:
    print("\nConclusion: Price series is STATIONARY (reject null hypothesis)")
else:
    print("\nConclusion: Price series is NON-STATIONARY (fail to reject null hypothesis)")

In [ ]:
# ACF and PACF plots
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

plot_acf(df['dam_price'].dropna(), lags=168, ax=axes[0])  # 1 week of hourly data
axes[0].set_title('Autocorrelation Function (ACF) - DAM Price', fontweight='bold')

plot_pacf(df['dam_price'].dropna(), lags=168, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF) - DAM Price', fontweight='bold')

plt.tight_layout()
plt.savefig('../results/figures/price_acf_pacf.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Key Insights <a id='insights'></a>

### Summary Dashboard

In [ ]:
# Create comprehensive dashboard
visualization.create_summary_dashboard(df)
plt.show()

### Key Findings

**Price Characteristics:**
1. Prices show strong hourly patterns (peak during day, low at night)
2. Seasonal variation with higher prices in summer/winter
3. Occasional price spikes indicating scarcity conditions
4. Strong autocorrelation suggests time series models will be effective

**Renewable Generation:**
1. Wind generation is relatively steady but with high variability
2. Solar follows clear diurnal pattern (zero at night)
3. Combined renewables contribute significant portion of supply

**Relationships:**
1. Positive correlation between load and price (demand driving prices)
2. Temperature affects both load (cooling/heating demand) and prices
3. Higher renewable penetration may be associated with lower prices (merit order effect)

**Implications for Modeling:**
1. Need to capture hourly and seasonal patterns
2. GARCH models appropriate for volatility clustering
3. Weather variables are important predictors
4. Must account for renewable generation uncertainty in portfolio optimization

---

## Next Steps

1. **Phase 2:** Build price forecasting models (GARCH, ARIMA, XGBoost, LSTM)
2. **Phase 3:** Model renewable generation with Monte Carlo simulation
3. **Phase 4:** Optimize portfolio allocation
4. **Phase 5:** Develop and backtest hedging strategies

---

**End of Notebook**

Save your work and proceed to `02_price_forecasting.ipynb`